<a href="https://colab.research.google.com/github/mherbert93/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module2-regression-2/LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [x] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [x] Engineer at least two new features. (See below for explanation & ideas.)
- [x] Fit a linear regression model with at least two features.
- [x] Get the model's coefficients and intercept.
- [x] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [x] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [x] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [3]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [0]:
coords = df[['latitude','longitude']]

miles_per_radian = 3959
epsilon = 0.08 / miles_per_radian
db = DBSCAN(eps=epsilon, min_samples=5, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_

df['labels'] = cluster_labels

In [0]:
df['created'] = pd.to_datetime(df['created'], infer_datetime_format=True) #convert to datatime format

In [7]:
df['interest_level'].value_counts()

low       33946
medium    11181
high       3690
Name: interest_level, dtype: int64

In [0]:
df['interest_level'] = df['interest_level'].astype('category')
df['interest_level'] = df['interest_level'].cat.codes #hot encode

In [0]:
#df['description'] = df['description'].fillna("")
#df['description_length'] = df['description'].str.len()

In [0]:
df['total_rooms'] = df['bathrooms'] + df['bedrooms']

In [0]:
def contains_outdoor_feature(row):
    if row['roof_deck'] == 1 or row['balcony'] == 1 or row['terrace'] == 1 or row['garden_patio'] == 1:
        val = 1
    else:
        val = 0
    return val

In [0]:
df['outdoor_feature'] = df.apply(contains_outdoor_feature, axis=1)

In [0]:
#subset = df[['latitude', 'longitude']]
#tuples = [tuple(x) for x in subset.values]

In [0]:
#type(tuples)

In [0]:
#import reverse_geocoder as rg

#coordinates = (tuples)

#results = rg.search(coordinates) # default mode = 2

#print(results)

In [0]:
#coordinates.dtype()

In [0]:
train = df[((df['created'].dt.month == 4) | (df['created'].dt.month == 5)) & (df['created'].dt.year == 2016)] #filter to april and may 2016
test = df[(df['created'].dt.month == 6) & (df['created'].dt.year == 2016)] #filter to june 2016

In [33]:
from sklearn.linear_model import LinearRegression

model = LinearRegression() #instaniate object

#features = ['bathrooms', 'bedrooms', 'fitness_center', 'dining_room', 'dishwasher', 'laundry_in_unit', 'balcony', 'elevator', 'cats_allowed', 'labels', 'doorman', 'interest_level', 'no_fee', 'longitude', 'hardwood_floors'] #define features, with added labels column
#features = ['bathrooms', 'bedrooms', 'fitness_center', 'dining_room', 'dishwasher', 'laundry_in_unit', 'outdoor_feature', 'elevator', 'cats_allowed', 'labels', 'doorman', 'interest_level', 'no_fee', 'longitude', 'hardwood_floors']
#features = ['bathrooms', 'bedrooms', 'labels', 'interest_level'] #define features, with added labels column
features = ['bathrooms','bedrooms', 'labels', 'dishwasher', 'laundry_in_unit', 'longitude', 'interest_level', 'elevator', 'doorman']
target = 'price' #define our target variable

model.fit(train[features], train[target]) #feed training data into linear regression model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [34]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

train_predict = model.predict(train[features])

print("MAE training accuracy: ", mean_absolute_error(train[target], train_predict))
print("RSME training accuracy: ", np.sqrt(mean_absolute_error(train[target], train_predict)))
print("R^2 training accuracy: ", r2_score(train[target], train_predict))

print("\n")

test_predict = model.predict(test[features])

print("MAE testing accuracy: ", mean_absolute_error(test[target], test_predict))
print("RSME testing accuracy: ", np.sqrt(mean_absolute_error(test[target], test_predict)))
print("R^2 testing accuracy: ", r2_score(test[target], test_predict))

guess = test[target].mean()
y_pred = [guess] * len(test[target])
baseline_mae = mean_absolute_error(test[target], y_pred)

print("\n")

print("MAE baseline testing accuracy: ", baseline_mae)

print("\n")

print('Intercept:', model.intercept_, "\n")
coefficients = pd.Series(model.coef_, features)
print("Model coefficients: \n")
print(coefficients.to_string())



MAE training accuracy:  697.7209321227963
RSME training accuracy:  26.414407661781784
R^2 training accuracy:  0.608464766288931


MAE testing accuracy:  708.8970294078932
RSME testing accuracy:  26.625120270299124
R^2 testing accuracy:  0.6149114366583925


MAE baseline testing accuracy:  1200.8695333832386


Intercept: -853252.4045255072 

Model coefficients: 

bathrooms           1813.705910
bedrooms             464.591584
labels                -2.179825
dishwasher           -89.362625
laundry_in_unit      436.545986
longitude         -11542.877847
interest_level      -141.055343
elevator              90.865132
doorman              454.569900


In [36]:
df.corr()

,bathrooms,bedrooms,latitude,longitude,price,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,labels,total_rooms,outdoor_feature
bathrooms,1.000000,0.524082,0.013407,-0.020099,0.687296,-0.009000,0.132882,0.021475,0.096922,0.024539,0.157173,0.172230,0.129547,-0.013202,0.148334,-0.016214,0.209366,0.102113,0.144450,0.233038,0.089073,0.136990,0.113315,0.066826,0.140767,-0.001944,0.014260,0.096283,0.066940,-0.003403,-0.026248,0.744468,0.162940
bedrooms,0.524082,1.000000,0.003490,0.055117,0.535503,0.028827,-0.024821,-0.011173,0.096420,-0.009750,-0.047562,0.152102,0.163000,0.008558,0.013150,0.001420,0.151790,0.045451,0.124912,0.189415,0.061006,0.097772,0.033441,0.000907,0.099149,-0.014880,-0.107341,0.070760,0.012306,0.003234,0.038956,0.958785,0.098107
latitude,0.013407,0.003490,1.000000,0.329185,-0.036286,-0.004656,-0.010523,-0.029808,0.018653,-0.030954,-0.043393,-0.020612,-0.026788,-0.041635,-0.108455,0.026802,-0.044339,-0.063198,-0.079919,0.017146,-0.030969,0.016877,0.027020,-0.056093,0.006466,-0.060054,-0.015650,-0.000589,-0.068330,-0.117199,-0.209451,0.007209,-0.043946
longitude,-0.020099,0.055117,0.329185,1.000000,-0.251004,-0.010981,-0.189836,-0.058475,-0.108493,-0.070329,-0.275734,-0.169220,-0.088033,-0.044562,-0.254960,0.000196,-0.130139,-0.161466,-0.098595,-0.024793,-0.125635,-0.035474,-0.075046,-0.108001,-0.049016,0.046755,-0.060018,-0.029846,-0.063635,-0.102955,0.306578,0.036503,-0.148636
price,0.687296,0.535503,-0.036286,-0.251004,1.000000,-0.032376,0.207169,0.051453,0.101503,0.060401,0.276215,0.223899,0.132240,-0.019417,0.228775,-0.029122,0.271195,0.122929,0.142146,0.242911,0.090269,0.139140,0.134513,0.071431,0.145973,-0.013251,0.007100,0.103672,0.072517,0.011517,-0.149903,0.649097,0.190657
interest_level,-0.009000,0.028827,-0.004656,-0.010981,-0.032376,1.000000,0.032143,-0.022568,0.096395,-0.022472,0.017152,0.080660,0.066947,0.006863,0.019271,0.004897,0.039712,0.044806,0.040090,0.030997,0.039409,0.017671,0.019363,0.021450,0.016929,-0.026081,0.013195,0.014188,0.009750,0.003730,0.003827,0.019596,0.049681
elevator,0.132882,-0.024821,-0.010523,-0.189836,0.207169,0.032143,1.000000,0.033347,0.270831,0.034833,0.614558,0.349832,0.227895,0.141097,0.431070,-0.097015,0.134158,0.332028,0.204343,0.200591,0.277666,0.168081,0.183664,0.184178,0.135329,0.025895,0.054918,0.084056,0.155396,0.114882,-0.115670,0.024873,0.352049
cats_allowed,0.021475,-0.011173,-0.029808,-0.058475,0.051453,-0.022568,0.033347,1.000000,-0.177633,0.937245,0.088480,-0.047880,-0.024052,0.105644,0.126886,0.047500,-0.008827,0.027590,0.066713,-0.022404,0.077759,0.019979,0.009704,0.042710,0.006152,0.031414,-0.037966,0.006690,0.039656,0.104496,0.031055,-0.001594,0.014861
hardwood_floors,0.096922,0.096420,0.018653,-0.108493,0.101503,0.096395,0.270831,-0.177633,1.000000,-0.185663,0.205119,0.634983,0.342971,-0.144728,0.167510,0.013435,0.360716,0.278727,0.188913,0.317281,0.237935,0.178089,0.170589,0.187635,0.181005,-0.194436,0.116572,0.161751,0.124829,-0.120489,-0.115486,0.107922,0.343106
dogs_allowed,0.024539,-0.009750,-0.030954,-0.070329,0.060401,-0.022472,0.034833,0.937245,-0.185663,1.000000,0.095434,-0.043839,-0.011414,0.093035,0.131521,0.051610,0.002510,0.034104,0.067859,-0.016154,0.089425,0.024472,0.009675,0.055655,0.005219,0.032729,-0.041014,0.009229,0.048123,0.106071,0.029999,0.000543,0.022399
